# Programmation côté serveur - 3ème partie

Dans cette dernière partie, nous abordons:

- exploiter un **formulaire** avec la méthode `POST`; application à la réalisation d'une **session** très simple (nous réutiliserons les cookies).

- **persister les données** côtés serveur; application à l'enregistrement de messages d'un utilisateur identifié.

## Formulaire soumis avec la méthode `POST`

La première méthode de soumission du formulaire modifie l'url et affiche en clair les valeurs envoyées ... cela n'est pas toujours souhaitable. Par exemple, imaginer le problème avec un formulaire d'authentification (login, mot de passe)...

On peut donc soumettre le formulaire avec `method="post"` de façon à ce que les données envoyées soient placées dans le **corps** de la requête du navigateur (elles sont toutefois envoyées en clair si on utilise pas http**s**...)

### Exemple

Le programme cgi <a href="demo_web_cgi/cgi-bin/form_post.py">form_post.py</a> (répertoire *demo_web_cgi/cgi-bin*) illustre la création d'une **session**. 

On appelle **session** l'ensemble des requête/réponse http d'un utilisateur «reconnue» par le serveur.

La **création d'une session** s'appuie sur l'**enregistrement d'un cookie** dont le rôle est d'identifier l'utilisateur en cours.

Dans cet exemple volontairement très simple, si l'utilisateur connaît le mot de passe du site (trouver le!), il peut s'authentifier et accéder à un contenu réservé aux utilisateur connectés. 

La logique de ce script est un peu plus élaborée que les précédents du fait que le programme peut s'exécuter dans plusieurs cas de figure:

- **si** l'utilisateur n'a pas de cookie de session: 
    - **si** le formulaire n'a pas été soumis: on lui affiche le formulaire,
    - **sinon si** (l'utilisateur vient de soumettre le formulaire) le mot de passe est correct:
        - on demande l'enregistrement d'un cookie et on affiche le contenu «secret»,
    - **sinon**: on réaffiche le formulaire avec un petit message d'erreur
- **sinon** (l'utilisateur a un cookie de session):
    - on récupère les informations de session (son login) en parsant le cookie,
    - et on affiche le message secret.

## Persistance des messages de l'utilisateur

Nous souhaitons à présent qu'un utilisateur authentifié puisse écrire des messages, se déconnecter, revenir plus tard et **récupérer** ses messages.

Pour parvenir à un tel résultat, il est nécessaire que le programme cgi enregistre les messages quelques part sur son hôte (la machine où il s'exécute).

**N'oublier pas** que le programme cgi s'exécute à chaque requête du client, il ne peut donc mémoriser des information dans une variable pour long terme.

On peut imaginer plusieurs façons de procéder pour enregistrer les informations, on peut le faire dans:
1. un **simple fichier texte** comme un fichier *csv* ou *json* par exemple,

2. un **fichier binaire** (non lisible) qui permet facilement de récupérer des données structurées (liste, dictionnaire, etc.),

3.  une **base de données** (programme de terminale) - seule solution possible pour un site qui permet d'enregistre beaucoup d'informations de natures différentes.


### Rappel python - les fichiers

Pour ouvrir un fichier texte avec Python, on utilise la fonction prédéfinie `open("nom_fichier", mode)` laquelle renvoie un «handler» de fichier qui sert à agir dessus.

Voilà à quoi cela ressemble basiquement, exemple d'un fichier ouvert en écriture (mode "w"):

```python
# éviter de procéder comme cela; utiliser with (voir plus loin)
f = open("mon_fichier", "w") 
f.write("...contenu...")
...
f.close() # libérer la ressource quand on a fini.
```

Le **mode** précise si le fichier est ouvert en lecture `"r"`, en écriture `"w"`, en mode «ajout» `"a"` (on peut les combiner par ex `"rw"`)

Par défaut, le fichier est ouvert en mode «texte». Pour un fichier **binaire**, on précise `b` au mode.

Après ouverture, le fichier (qui est sur le disque) est placé en mémoire centrale dans une zone spéciale appelée tampon *buffer* (par le système d'exploitation); il est indispensable d'indiquer la fin de l'utilisation du fichier par `.close()` qui indique au système d'exploitation de libérer cette zone mémoire.

Pour rendre ce processus plus net pour le programmeur, python dispose d'un mot clé spécial `with` (on appelle cela un manager de contexte). L'utilisation d'un fichier sera alors de la forme:

```python
with open("mon_fichier", "w") as f:
    # bloc d'accès au fichier via f
    f.write("...contenu...")
    ...
    # à la sortie du bloc, le fichier est libéré: inutile de penser à close

# dans cette zone, le fichier a été libéré.
```

Voici un exemple (attention, pour le tester, il faut copier ce notebook dans votre espace personnel):

In [ ]:
descripteurs = ["nom", "prenom", "email"]
valeurss = [
    ["Dupond", "tartampion", "d.tart@truc.com"],
    ["Ulrich", "Yahn", "champion@world.org"],
]

with open("personnes_importantes", "w") as f:
    f.write(
        ",".join(descripteurs) + "\n"
    )
    for valeurs in valeurss:
        f.write(
            ",".join(valeurs) + "\n"
        )

Après exécution de cette cellule, vous devriez apercevoir le fichier créé (sinon c'est que ce notebook n'est pas dans votre répertoire personnelle).

Ajoutons une ligne, puis relisons le tout:

In [ ]:
with open("personnes_importantes", "a") as f:
    f.write("Doe, John, john.doe@python.org\n")

with open("personnes_importantes", "r") as f:
    for ligne in f:
        print(ligne, end="")

print("fin de la démo")

Observer ce qui se passe lorque vous exécuter plusieurs fois la dernière cellule... comprenez-vous?

Dans le contexte d'une application web, la difficulté avec cette manière de faire est qu'il est nécessaire de formater l'information dans le sens de l'écriture puis de décrypter ce format dans le sens de la lecture.

Heureusement, Python propose des outils afin de sauvegarder/restaurer ses propres structures (liste, dictionnaire, ...)

### Sauvegarder/Restaurer des données avec le module `pickle`

Ce module, propre à Python, utilise un fichier binaire comme support de la sauvegarde.

Voici comment il s'utilise dans les grandes lignes (sauvegarder -> `dump`; restaurer -> `load`):

```python
import pickle

# sauvegarde d'une donnée structurée (qui peut être de type list, dict, set, ...)
with open("sauvegarde", "wb") as f:
    # sauvegarde d'une structure référencée par variable
    pickle.dump(variable, f)

# restauration
with open("sauvegarde", "rb") as f:
    variable = pickle.load(f)
```

### Exemple final

Le programme cgi <a href="demo_web_cgi/cgi-bin/messages.py">messages.py</a> (répertoire *demo_web_cgi/cgi-bin*) permet à l'utilisateur authentifié d'écrire des messages sauvegardés par l'application. 

Il utilise quelques éléments que nous n'avons pas encore vue:

- **Redirection**: Pour éviter que le code soit trop long, on réutilise le programme        d'authentification *form_post.py* en utilisant une redirection à l'aide d'une balise d'entête HTML de   la forme:

  ```html
  <meta http-equiv="refresh" 
        content="<nb_secondes_attente>;URL=<chemin_prog_cgi>"
  />
  ```
  Son effet est d'attendre le temps indiqué puis de demander la ressource précisée par URL.

- **Gestion des cookie**: nous avons vu que décrypter un cookie n'était pas si simple. Heureusement le sous-module `cookies` de `http` permet de faire cela plus simplement:

  ```python
  import os
  from http import cookies
  
  # récupération du cookie
  cookie = cookies.SimpleCookie(
      os.environ("HTTP_COOKIE")
  )
  
  # extraction de la valeur associée à une clé
  valeur_de_cle = cookie["<la clef>"].value
  ```

- **Tentative de lecture d'un fichier inexistant**: la première fois que la page d'écriture des messages est visitée, le fichier qui les contient n'existe pas.

  Plutôt que de détecter si c'est la première fois que la page est visitée, on utilise un structure
  permettant d'éviter que le programme ne s'arrête en cas d'erreur (tentative d'ouvrir un fichier non   existant). Voilà à quoi cette structure ressemble.

  ```python
  try:
      # code risquant de produire une erreur (on dit exception)
  except:
      # code à appliquer si l'erreur se produit
  ```

### Vidéo d'aide

In [ ]:
%%HTML
<iframe src="https://player.vimeo.com/video/403634384" width="640" height="564" frameborder="0" allow="autoplay; fullscreen" allowfullscreen></iframe>

## Conclusion - perspective

La programmation web côté serveur fait intervenir beaucoup de connaissances diverses et est donc difficile pour un débutant. 

L'informatique ressemble à un gigantesque puzzle dont on prend connaissance pièce par pièce sans pour autant parvenir à voir le paysage dans son ensemble: mais rassurez-vous, petit à petit les pièces s'assemblent, le tout prend corps et cela va de plus en plus vite car les même techniques sont utilisées encore et encore. Patience donc.



Néanmoins, si vous êtes parvenu jusqu'ici et avez le sentiment de comprendre les programmes dans les grandes lignes (sans pour autant être en mesure de les écrire par vous-même), vous avez acquis l'essentiel.

Pour réaliser des applications web de grande envergure, on utilise en pratique des **frameworks**. Ce terme regroupe à la fois:

- une méthodologie de codage propre au framework utilisé,
- des librairies pour automatiser les tâches ingrates (gestion des sessions, persistance des données, etc.)

Il y a de très nombreux **frameworks** pour différents usages et différents langages.

Pour ce qui concerne Python, il y a notamment:
- [flask](https://palletsprojects.com/p/flask/) qui regroupe en fait une palette d'outils pour le web,
- [django](https://www.djangoproject.com/) très puissant et très utilisé,
- et la liste ne s'arrête pas là ...

**Pourquoi ne pas avoir appris l'un d'eux alors?** Eh bien ... il y a deux raisons principales:
1. ces outils font appels à des notions de programmation avancées (packetage, classes, structures, base de données...),
2. ils cachent les détails du dialogue http, tout en les utilisant intensivement. Ils présupposent que leurs utilisateurs connaissent les mécanismes de base du web et s'appuient dessus.

### Mini-projet - pour faire le point sur toutes les notions vues jusqu'ici

Avec les connaissances acquises (ou en cours d'acquisition...), vous devriez être en mesure de réaliser une petite «chatroom»:

- Les utilisateurs peuvent s'authentifier,
- ils postent des messages et voient les messages des autres utilisateurs «en ligne».

Cela n'a pas besoin d'être beau, mais simplement fonctionnel: un seul programme cgi suffit.

Cependant, le serveur cgi de python (lancé en ligne de commande) a le défaut d'exiger que les scripts soit dans un sous dossier *cgi-bin* ce qui n'est pas pratique pour une telle application «mono-script».

Voici le code d'un serveur cgi qui lance le (ou les scripts) situés dans le répertoire où on le lance:

```python
#!/opt/tljh/user/bin/python

import sys
import http.server as s
 
port = sys.argv[1]

gestionnaire = s.CGIHTTPRequestHandler
gestionnaire.cgi_directories = ["/"]

httpd = s.HTTPServer(
    ("", int(port)),
    gestionnaire
)
print("Serveur actif sur le port :", port)
httpd.serve_forever()
```

Créer un fichier *httpd_cgi* avec ce code, rendez le exécutable et placer le dans votre dossier `~/bin` pour en faire une commande (comme expliqué dans [cette vidéo](https://vimeo.com/398805308)).

Créer un dossier *web_mini_chat* et dedans, votre programme cgi *minichat.py* (à rendre exécutable) dont le début pourrait être:

```python
#!/opt/tljh/user/bin/python

import cgitb; cgitb.enable() # facilite le débogage en cours de développement

print("Content-type: text/html; charset=utf-8/n")
print("<h2>Mini-Chat: c'est parti!</h2>")
```

Cela fait, commencer votre développement en lançant votre serveur `$ httpd_cgi <port>` **depuis** le dossier *web_mini_chat* et en ouvrant l'url `jhub.fdex.eu:<port>/minichat.py`.
